# Fibonacci Retracement in Python for trading

The Fibonacci Retracement is an analysis on the Fibonacci sequence and applies key ratios to determine where price corrections are likely to occurr. Usually this method helps traders decide when to take positions, it only helps with the entry point decisions, so it doesn't guess the trend nor the future price, but it's a help in trending markets. 

On a trending market, if we can identify two pivot points or fractal points and we already know the trend direction. The challenge is to know when to execute our position, when to enter the market. So for example, if we already know the trend is up and we're going long we want to buy, but we also need to minimize our risk and maximize our profit. This is where Fibonacci can be a helpful approach: we can identify the Fibonacci levels and wait for the price to retrace back to the golden zone, which is somewhere around 60% retracement distance.Sometimes 60% works fine and some other times the price might go back to as low as 70% retracement. As long as WE stick to a systematic approach, WE should be able to make profit and decrease your risk. 

The reason this method works is mainly the reward risk ratio, which is usually above 1.5, going in the direction of the trend.

Let's see this strategy in an example:


In [4]:
# Loading the data
!pip install pandas_ta

import pandas as pd
import pandas_ta as ta

# Reading the data
df = pd.read_csv("EURUSD_Candlestick_1_Hour_BID_04.05.2003-15.04.2023.csv")

# Evaluating NA values in df
df=df[df['volume']!=0]
df.reset_index(drop=True, inplace= True)
df.isna().sum()
df['RSI'] = ta.rsi(df.close, lenght= 12)
df['EMA'] = ta.ema(df.close, lenght=150)
df.tail()

df=df[0:2000]

Defaulting to user installation because normal site-packages is not writeable
